# Week 6: Искусственные нейронные сети (ANN)
В данной работе мы изучим:

1. **Функции активации** - математические основы, свойства и области применения
2. **Архитектуры ANN** - принципы построения полносвязных нейронных сетей
3. **Forward и Backward pass** - прямое и обратное распространение в нейронных сетях
4. **Backpropagation** - алгоритм обратного распространения ошибки
5. **PyTorch implementation** - практическая реализация нейронных сетей
6. **Обучение на реальных данных** - применение ANN для классификации изображений

## 1. Импорт библиотек и настройка окружения

Для работы с нейронными сетями нам понадобятся следующие библиотеки:
- **NumPy** - для математических операций с массивами
- **PyTorch** - современный фреймворк для глубокого обучения
- **Plotly** - для создания интерактивных визуализаций
- **torchvision** - для работы с компьютерным зрением
- **ipywidgets** - для создания интерактивных элементов

In [1]:
# Импорт основных библиотек
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# PyTorch библиотеки
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

# Для интерактивных элементов
from ipywidgets import interact, interact_manual
import ipywidgets as widgets

# Настройка для воспроизводимости результатов
np.random.seed(42)
torch.manual_seed(42)

# Настройка отображения графиков
plt.style.use('default')

print("Все библиотеки успешно импортированы!")
print(f"PyTorch версия: {torch.__version__}")
print(f"NumPy версия: {np.__version__}")

# Проверка доступности GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

Все библиотеки успешно импортированы!
PyTorch версия: 2.7.1
NumPy версия: 1.26.4
Используемое устройство: cpu


## 2. Функции активации: теория и математическое описание

### Что такое функция активации?

**Функция активации** - это математическая функция, которая определяет выходной сигнал нейрона на основе входного сигнала. Она играет ключевую роль в способности нейронной сети моделировать сложные нелинейные зависимости.

### Математическое определение

Для нейрона с входами $x_1, x_2, ..., x_n$, весами $w_1, w_2, ..., w_n$ и смещением $b$, функция активации $f$ применяется к взвешенной сумме входов:

$$z = \sum_{i=1}^{n} w_i x_i + b$$

$$a = f(z)$$

где:
- $z$ - предактивация (pre-activation)
- $a$ - активация (activation)
- $f$ - функция активации

### Зачем нужны функции активации?

**Интуиция**: Без функций активации нейронная сеть представляла бы собой просто последовательность линейных преобразований, что эквивалентно одному линейному преобразованию. Функции активации вносят нелинейность, позволяя сети аппроксимировать любые сложные функции.

### Требования к функциям активации

1. **Нелинейность** - для моделирования сложных зависимостей
2. **Дифференцируемость** - для применения алгоритма backpropagation
3. **Монотонность** (желательно) - для стабильности обучения
4. **Вычислительная эффективность** - для быстрого обучения

### Основные функции активации

#### 1. Sigmoid (Логистическая функция)

**Математическое определение:**
$$\sigma(z) = \frac{1}{1 + e^{-z}}$$

**Производная:**
$$\sigma'(z) = \sigma(z)(1 - \sigma(z))$$

**Свойства:**
- Область значений: $(0, 1)$
- Гладкая S-образная кривая
- Интерпретируется как вероятность

**Плюсы:**
- Плавный переход между 0 и 1
- Хорошая интерпретация для бинарной классификации
- Дифференцируемая на всей области определения

**Минусы:**
- **Проблема исчезающих градиентов** - градиенты становятся очень малыми при больших $|z|$
- **Не центрирована относительно нуля** - выходы всегда положительные
- **Вычислительно затратная** - требует вычисления экспоненты

**Когда использовать:**
- Выходной слой для бинарной классификации
- Исторические архитектуры (сейчас редко)

#### 2. Tanh (Гиперболический тангенс)

**Математическое определение:**
$$\tanh(z) = \frac{e^z - e^{-z}}{e^z + e^{-z}} = \frac{2}{1 + e^{-2z}} - 1$$

**Производная:**
$$\tanh'(z) = 1 - \tanh^2(z)$$

**Свойства:**
- Область значений: $(-1, 1)$
- Центрирована относительно нуля
- S-образная кривая

**Плюсы:**
- Центрирована относительно нуля (лучше для скрытых слоев)
- Более крутые градиенты, чем sigmoid

**Минусы:**
- Все еще проблема исчезающих градиентов
- Вычислительно затратная

**Когда использовать:**
- Скрытые слои в простых сетях
- LSTM и RNN ячейки

#### 3. ReLU (Rectified Linear Unit)

**Математическое определение:**
$$\text{ReLU}(z) = \max(0, z) = \begin{cases} 
z & \text{если } z \gt 0 \\
0 & \text{если } z \leq 0
\end{cases}$$

**Производная:**
$$\text{ReLU}'(z) = \begin{cases} 
1 & \text{если } z \gt 0 \\
0 & \text{если } z \leq 0
\end{cases}$$

**Свойства:**
- Область значений: $[0, +\infty)$
- Линейная для положительных значений
- Разреженная активация (много нулей)

**Плюсы:**
- **Простота вычислений** - очень быстрая
- **Решает проблему исчезающих градиентов** для положительных значений
- **Разреженность** - улучшает эффективность обучения
- **Биологическая мотивация** - похожа на активацию реальных нейронов

**Минусы:**
- **Проблема умирающих нейронов** - нейроны с отрицательными входами перестают обучаться
- **Не дифференцируемая в нуле** (на практике не критично)
- **Неограниченная сверху** - может привести к взрывающимся градиентам

**Когда использовать:**
- Де-факто стандарт для скрытых слоев
- Сверточные нейронные сети
- Большинство современных архитектур

#### 4. Leaky ReLU

**Математическое определение:**
$$\text{LeakyReLU}(z) = \begin{cases} 
z & \text{если } z \gt 0 \\
\alpha z & \text{если } z \leq 0
\end{cases}$$

где $\alpha$ - малый положительный параметр (обычно 0.01)

**Производная:**
$$\text{LeakyReLU}'(z) = \begin{cases} 
1 & \text{если } z \gt 0 \\
\alpha & \text{если } z \leq 0
\end{cases}$$

**Плюсы:**
- Решает проблему умирающих нейронов
- Сохраняет вычислительную простоту ReLU

**Минусы:**
- Дополнительный гиперпараметр $\alpha$
- Эмпирические преимущества не всегда значительны

### Пример ручного расчета

Рассмотрим простой пример с одним нейроном:
- Входы: $x_1 = 2, x_2 = -1$
- Веса: $w_1 = 0.5, w_2 = -0.3$  
- Смещение: $b = 0.1$

**Шаг 1:** Вычисляем предактивацию
$$z = w_1x_1 + w_2x_2 + b = 0.5 \cdot 2 + (-0.3) \cdot (-1) + 0.1 = 1.0 + 0.3 + 0.1 = 1.4$$

**Шаг 2:** Применяем различные функции активации

**Sigmoid:**
$$\sigma(1.4) = \frac{1}{1 + e^{-1.4}} = \frac{1}{1 + 0.247} = \frac{1}{1.247} \approx 0.802$$

**Tanh:**
$$\tanh(1.4) = \frac{e^{1.4} - e^{-1.4}}{e^{1.4} + e^{-1.4}} = \frac{4.055 - 0.247}{4.055 + 0.247} = \frac{3.808}{4.302} \approx 0.885$$

**ReLU:**
$$\text{ReLU}(1.4) = \max(0, 1.4) = 1.4$$

**Leaky ReLU** (при $\alpha = 0.01$):
$$\text{LeakyReLU}(1.4) = 1.4$$
(так как $1.4 \gt 0$)

### Выбор функции активации

**Рекомендации по слоям:**

**Скрытые слои:**
- ReLU, Leaky ReLU
- Причина: Быстрые, решают проблему исчезающих градиентов

**Выходной слой (бинарная классификация):**
- Sigmoid
- Причина: Выход интерпретируется как вероятность

**Выходной слой (многоклассовая классификация):**
- Softmax
- Причина: Сумма выходов равна 1

**Выходной слой (регрессия):**
- Линейная (без активации)
- Причина: Неограниченные значения

## 3. Реализация функций активации

In [2]:
def sigmoid(z):
    """
    Сигмоидная функция активации.
    
    Parameters
    ----------
    z : array_like
        Входные значения (предактивация)
        
    Returns
    -------
    array_like
        Значения после применения сигмоидной функции, в диапазоне (0, 1)
        
    Notes
    -----
    Использует стабильную реализацию для избежания переполнения:
    - Для z > 0: sigmoid(z) = 1 / (1 + exp(-z))
    - Для z <= 0: sigmoid(z) = exp(z) / (1 + exp(z))
    """
    z = np.array(z, dtype=np.float64)
    # Стабильная реализация для избежания переполнения
    positive_mask = z > 0
    result = np.zeros_like(z)
    
    # Для положительных z
    result[positive_mask] = 1 / (1 + np.exp(-z[positive_mask]))
    
    # Для отрицательных z
    negative_mask = ~positive_mask
    exp_z = np.exp(z[negative_mask])
    result[negative_mask] = exp_z / (1 + exp_z)
    
    return result

def sigmoid_derivative(z):
    """
    Производная сигмоидной функции.
    
    Parameters
    ----------
    z : array_like
        Входные значения
        
    Returns
    -------
    array_like
        Значения производной сигмоидной функции
    """
    s = sigmoid(z)
    return s * (1 - s)

# Тестирование функций
test_values = np.array([-5, -2, 0, 2, 5])
print("Тестирование sigmoid:")
print(f"Входные значения: {test_values}")
print(f"Sigmoid: {sigmoid(test_values)}")
print(f"Производная: {sigmoid_derivative(test_values)}")
print()

Тестирование sigmoid:
Входные значения: [-5 -2  0  2  5]
Sigmoid: [0.00669285 0.11920292 0.5        0.88079708 0.99330715]
Производная: [0.00664806 0.10499359 0.25       0.10499359 0.00664806]



In [3]:
def tanh(z):
    """
    Гиперболический тангенс.
    
    Parameters
    ----------
    z : array_like
        Входные значения (предактивация)
        
    Returns
    -------
    array_like
        Значения после применения tanh, в диапазоне (-1, 1)
    """
    return np.tanh(z)

def tanh_derivative(z):
    """
    Производная функции tanh.
    
    Parameters
    ----------
    z : array_like
        Входные значения
        
    Returns
    -------
    array_like
        Значения производной tanh
    """
    return 1 - np.tanh(z)**2

# Тестирование tanh
print("Тестирование tanh:")
print(f"Входные значения: {test_values}")
print(f"Tanh: {tanh(test_values)}")
print(f"Производная: {tanh_derivative(test_values)}")
print()

Тестирование tanh:
Входные значения: [-5 -2  0  2  5]
Tanh: [-0.9999092  -0.96402758  0.          0.96402758  0.9999092 ]
Производная: [1.81583231e-04 7.06508249e-02 1.00000000e+00 7.06508249e-02
 1.81583231e-04]



In [4]:
def relu(z):
    """
    Rectified Linear Unit (ReLU) функция активации.
    
    Parameters
    ----------
    z : array_like
        Входные значения (предактивация)
        
    Returns
    -------
    array_like
        Значения после применения ReLU: max(0, z)
    """
    return np.maximum(0, z)

def relu_derivative(z):
    """
    Производная функции ReLU.
    
    Parameters
    ----------
    z : array_like
        Входные значения
        
    Returns
    -------
    array_like
        Значения производной ReLU: 1 если z > 0, иначе 0
        
    Notes
    -----
    В точке z=0 производная технически не определена, но на практике
    принимается равной 0.
    """
    return (z > 0).astype(float)

# Тестирование ReLU
print("Тестирование ReLU:")
print(f"Входные значения: {test_values}")
print(f"ReLU: {relu(test_values)}")
print(f"Производная: {relu_derivative(test_values)}")
print()

Тестирование ReLU:
Входные значения: [-5 -2  0  2  5]
ReLU: [0 0 0 2 5]
Производная: [0. 0. 0. 1. 1.]



In [5]:
def leaky_relu(z, alpha=0.01):
    """
    Leaky ReLU функция активации.
    
    Parameters
    ----------
    z : array_like
        Входные значения (предактивация)
    alpha : float, default=0.01
        Коэффициент для отрицательных значений
        
    Returns
    -------
    array_like
        Значения после применения Leaky ReLU
    """
    return np.where(z > 0, z, alpha * z)

def leaky_relu_derivative(z, alpha=0.01):
    """
    Производная функции Leaky ReLU.
    
    Parameters
    ----------
    z : array_like
        Входные значения
    alpha : float, default=0.01
        Коэффициент для отрицательных значений
        
    Returns
    -------
    array_like
        Значения производной Leaky ReLU
    """
    return np.where(z > 0, 1, alpha)

# Тестирование Leaky ReLU
print("Тестирование Leaky ReLU:")
print(f"Входные значения: {test_values}")
print(f"Leaky ReLU: {leaky_relu(test_values)}")
print(f"Производная: {leaky_relu_derivative(test_values)}")
print()

Тестирование Leaky ReLU:
Входные значения: [-5 -2  0  2  5]
Leaky ReLU: [-0.05 -0.02  0.    2.    5.  ]
Производная: [0.01 0.01 0.01 1.   1.  ]



## 4. Визуализация функций активации

In [6]:
# Создание данных для визуализации
z_range = np.linspace(-5, 5, 1000)

# Вычисление значений функций активации
sigmoid_values = sigmoid(z_range)
tanh_values = tanh(z_range)
relu_values = relu(z_range)
leaky_relu_values = leaky_relu(z_range)

# Создание интерактивного графика функций активации
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Sigmoid', 'Tanh', 'ReLU', 'Leaky ReLU'],
    vertical_spacing=0.15,  # Увеличиваем вертикальный отступ
    horizontal_spacing=0.12  # Увеличиваем горизонтальный отступ
)

# Sigmoid
fig.add_trace(
    go.Scatter(x=z_range, y=sigmoid_values, 
               name='Sigmoid', line=dict(color='blue', width=3)),
    row=1, col=1
)

# Tanh
fig.add_trace(
    go.Scatter(x=z_range, y=tanh_values, 
               name='Tanh', line=dict(color='red', width=3)),
    row=1, col=2
)

# ReLU
fig.add_trace(
    go.Scatter(x=z_range, y=relu_values, 
               name='ReLU', line=dict(color='green', width=3)),
    row=2, col=1
)

# Leaky ReLU
fig.add_trace(
    go.Scatter(x=z_range, y=leaky_relu_values, 
               name='Leaky ReLU', line=dict(color='orange', width=3)),
    row=2, col=2
)

# Настройка осей и внешнего вида
for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(
            title_text="Предактивация (z)", 
            row=i, col=j, 
            showgrid=True, 
            gridwidth=1, 
            gridcolor='lightgray',
            title_font=dict(size=10)  # Уменьшаем размер шрифта подписей осей
        )
        fig.update_yaxes(
            title_text="Активация f(z)", 
            row=i, col=j,
            showgrid=True, 
            gridwidth=1, 
            gridcolor='lightgray',
            title_font=dict(size=10)  # Уменьшаем размер шрифта подписей осей
        )

fig.update_layout(
    title_text="Функции активации: сравнение форм",
    title_x=0.5,
    title_font=dict(size=16),  # Размер заголовка
    height=650,  # Увеличиваем высоту
    showlegend=False,
    font=dict(size=10),  # Уменьшаем общий размер шрифта
    margin=dict(t=80, b=50, l=50, r=50)  # Добавляем отступы
)

# Обновляем подзаголовки
fig.update_annotations(font_size=12)  # Размер подзаголовков

fig.show()

In [7]:
# Визуализация производных функций активации
sigmoid_deriv = sigmoid_derivative(z_range)
tanh_deriv = tanh_derivative(z_range)
relu_deriv = relu_derivative(z_range)
leaky_relu_deriv = leaky_relu_derivative(z_range)

# Создание графика производных
fig_deriv = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Производная Sigmoid', 'Производная Tanh', 
                   'Производная ReLU', 'Производная Leaky ReLU'],
    vertical_spacing=0.15,  # Увеличиваем вертикальный отступ
    horizontal_spacing=0.12  # Увеличиваем горизонтальный отступ
)

# Производная Sigmoid
fig_deriv.add_trace(
    go.Scatter(x=z_range, y=sigmoid_deriv, 
               name="Sigmoid'", line=dict(color='blue', width=3)),
    row=1, col=1
)

# Производная Tanh
fig_deriv.add_trace(
    go.Scatter(x=z_range, y=tanh_deriv, 
               name="Tanh'", line=dict(color='red', width=3)),
    row=1, col=2
)

# Производная ReLU
fig_deriv.add_trace(
    go.Scatter(x=z_range, y=relu_deriv, 
               name="ReLU'", line=dict(color='green', width=3)),
    row=2, col=1
)

# Производная Leaky ReLU
fig_deriv.add_trace(
    go.Scatter(x=z_range, y=leaky_relu_deriv, 
               name="Leaky ReLU'", line=dict(color='orange', width=3)),
    row=2, col=2
)

# Настройка осей
for i in range(1, 3):
    for j in range(1, 3):
        fig_deriv.update_xaxes(
            title_text="Предактивация (z)", 
            row=i, col=j,
            showgrid=True, 
            gridwidth=1, 
            gridcolor='lightgray',
            title_font=dict(size=10)  # Уменьшаем размер шрифта подписей осей
        )
        fig_deriv.update_yaxes(
            title_text="Производная f'(z)", 
            row=i, col=j,
            showgrid=True, 
            gridwidth=1, 
            gridcolor='lightgray',
            title_font=dict(size=10)  # Уменьшаем размер шрифта подписей осей
        )

fig_deriv.update_layout(
    title_text="Производные функций активации",
    title_x=0.5,
    title_font=dict(size=16),  # Размер заголовка
    height=650,  # Увеличиваем высоту
    showlegend=False,
    font=dict(size=10),  # Уменьшаем общий размер шрифта
    margin=dict(t=80, b=50, l=50, r=50)  # Добавляем отступы
)

# Обновляем подзаголовки
fig_deriv.update_annotations(font_size=12)  # Размер подзаголовков

fig_deriv.show()

## 5. Интерактивное сравнение функций активации

Этот интерактивный виджет позволяет экспериментировать с различными функциями активации и наблюдать, как они влияют на форму выходного сигнала.

In [ ]:
def interactive_activation_comparison(function_1='sigmoid', function_2='relu', 
                                    z_min=-5, z_max=5, alpha=0.01):
    """
    Интерактивное сравнение двух функций активации.
    
    Parameters
    ----------
    function_1, function_2 : str
        Названия функций активации для сравнения
    z_min, z_max : float
        Диапазон значений для визуализации
    alpha : float
        Параметр для Leaky ReLU
    """
    z = np.linspace(z_min, z_max, 1000)
    
    # Словарь функций
    functions = {
        'sigmoid': sigmoid,
        'tanh': tanh,
        'relu': relu,
        'leaky_relu': lambda x: leaky_relu(x, alpha)
    }
    
    derivatives = {
        'sigmoid': sigmoid_derivative,
        'tanh': tanh_derivative,
        'relu': relu_derivative,
        'leaky_relu': lambda x: leaky_relu_derivative(x, alpha)
    }
    
    # Вычисление значений
    y1 = functions[function_1](z)
    y2 = functions[function_2](z)
    dy1 = derivatives[function_1](z)
    dy2 = derivatives[function_2](z)
    
    # Создание графика
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=[f'Функции активации', f'Производные'],
        horizontal_spacing=0.15
    )
    
    # Функции активации
    fig.add_trace(
        go.Scatter(x=z, y=y1, name=function_1.replace('_', ' ').title(),
                  line=dict(color='blue', width=3)),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=z, y=y2, name=function_2.replace('_', ' ').title(),
                  line=dict(color='red', width=3)),
        row=1, col=1
    )
    
    # Производные
    fig.add_trace(
        go.Scatter(x=z, y=dy1, name=f"{function_1.replace('_', ' ').title()}'",
                  line=dict(color='blue', width=3, dash='dash'), showlegend=False),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Scatter(x=z, y=dy2, name=f"{function_2.replace('_', ' ').title()}'",
                  line=dict(color='red', width=3, dash='dash'), showlegend=False),
        row=1, col=2
    )
    
    # Настройка осей
    fig.update_xaxes(title_text="Предактивация (z)", row=1, col=1)
    fig.update_yaxes(title_text="Активация f(z)", row=1, col=1)
    fig.update_xaxes(title_text="Предактивация (z)", row=1, col=2)
    fig.update_yaxes(title_text="Производная f'(z)", row=1, col=2)
    
    fig.update_layout(
        title_text=f"Сравнение: {function_1.replace('_', ' ').title()} vs {function_2.replace('_', ' ').title()}",
        height=400,
        legend=dict(x=0.02, y=0.98)
    )
    
    fig.show()

# Создание интерактивного виджета
interact(interactive_activation_comparison,
         function_1=['sigmoid', 'tanh', 'relu', 'leaky_relu'],
         function_2=['sigmoid', 'tanh', 'relu', 'leaky_relu'],
         z_min=widgets.FloatSlider(value=-5, min=-10, max=0, step=0.5, description='z_min:'),
         z_max=widgets.FloatSlider(value=5, min=0, max=10, step=0.5, description='z_max:'),
         alpha=widgets.FloatSlider(value=0.01, min=0.001, max=0.1, step=0.001, description='alpha:'));

interactive(children=(Dropdown(description='function_1', options=('sigmoid', 'tanh', 'relu', 'leaky_relu'), va…

## 6. Архитектуры нейронных сетей: теория

### Что такое архитектура нейронной сети?

**Архитектура нейронной сети** определяет структурную организацию нейронов: количество слоев, количество нейронов в каждом слое, способы их соединения и типы функций активации.

### Основные компоненты ANN

#### 1. Нейрон (Perceptron)

**Математическая модель нейрона:**

Для нейрона с входами $\mathbf{x} = [x_1, x_2, ..., x_n]^T$, весами $\mathbf{w} = [w_1, w_2, ..., w_n]^T$ и смещением $b$:

$$z = \mathbf{w}^T\mathbf{x} + b = \sum_{i=1}^{n} w_i x_i + b$$

$$a = f(z)$$

где $f$ - функция активации.

**Интуиция:** Нейрон принимает решение на основе взвешенной суммы входов. Веса определяют важность каждого входа, смещение сдвигает порог активации.

#### 2. Слои (Layers)

**Входной слой (Input Layer):**
- Принимает исходные данные
- Количество нейронов = количеству признаков
- Обычно без функции активации

**Скрытые слои (Hidden Layers):**
- Выполняют основную обработку информации
- Извлекают и комбинируют признаки
- Обычно используют нелинейные функции активации

**Выходной слой (Output Layer):**
- Производит финальные предсказания
- Количество нейронов зависит от задачи:
  - 1 нейрон для бинарной классификации
  - K нейронов для K-классовой классификации
  - Произвольное количество для регрессии

### Типы архитектур

#### 1. Многослойный персептрон (MLP - Multi-Layer Perceptron)

**Определение:** Полносвязная сеть, где каждый нейрон одного слоя соединен со всеми нейронами следующего слоя.

**Математическое описание:**

Для сети с L слоями:
- $\mathbf{a}^{(0)} = \mathbf{x}$ (входной слой)
- $\mathbf{z}^{(l)} = \mathbf{W}^{(l)}\mathbf{a}^{(l-1)} + \mathbf{b}^{(l)}$ для $l = 1, 2, ..., L$
- $\mathbf{a}^{(l)} = f^{(l)}(\mathbf{z}^{(l)})$ для $l = 1, 2, ..., L$

где:
- $\mathbf{W}^{(l)} \in \mathbb{R}^{n_l \times n_{l-1}}$ - матрица весов слоя $l$
- $\mathbf{b}^{(l)} \in \mathbb{R}^{n_l}$ - вектор смещений слоя $l$
- $n_l$ - количество нейронов в слое $l$
- $f^{(l)}$ - функция активации слоя $l$

**Плюсы MLP:**
- **Универсальная аппроксимация** - может аппроксимировать любую непрерывную функцию
- **Простота реализации** - стандартная архитектура
- **Эффективность обучения** - хорошо работает с backpropagation
- **Гибкость** - подходит для различных задач

**Минусы MLP:**
- **Много параметров** - может привести к переобучению
- **Не учитывает структуру данных** - не подходит для изображений, текстов
- **Вычислительная сложность** - O(n²) связей между слоями

**Когда использовать:**
- Табличные данные
- Задачи классификации и регрессии
- Когда нет специфической структуры в данных

#### 2. Глубокие нейронные сети (Deep Neural Networks)

**Определение:** Сети с большим количеством скрытых слоев (обычно > 3).

**Преимущества глубины:**
- **Иерархическое представление** - каждый слой извлекает признаки более высокого уровня
- **Композиционность** - сложные функции как композиция простых
- **Эффективность представления** - экспоненциальное увеличение выразительности

**Проблемы глубоких сетей:**
- **Исчезающие градиенты** - градиенты уменьшаются при распространении назад
- **Взрывающиеся градиенты** - градиенты растут экспоненциально
- **Переобучение** - больше параметров = больше риск переобучения

### Выбор архитектуры

#### Количество слоев

| Количество слоев | Тип задач | Характеристики |
|------------------|-----------|----------------|
| 1 (Linear) | Линейная регрессия, простая классификация | Только линейные зависимости |
| 2-3 | Простые нелинейные задачи | XOR, простая классификация |
| 4-10 | Стандартные задачи ML | Большинство табличных данных |
| 10+ | Сложные задачи | Изображения, текст, речь |

#### Количество нейронов в слое

**Эмпирические правила:**
- Размер скрытого слоя: между размером входного и выходного слоев
- Правило 2/3: $n_{hidden} = \frac{2}{3}(n_{input} + n_{output})$
- Правило квадратного корня: $n_{hidden} = \sqrt{n_{input} \times n_{output}}$

### Пример ручного расчета для простой сети

Рассмотрим сеть 2-3-1 (2 входа, 3 нейрона в скрытом слое, 1 выход):

**Дано:**
- Входы: $x_1 = 0.5, x_2 = -0.3$
- Веса первого слоя: 
  $$\mathbf{W}^{(1)} = \begin{pmatrix} 0.2 & 0.4 \\ -0.1 & 0.3 \\ 0.6 & -0.2 \end{pmatrix}$$
- Смещения первого слоя: $\mathbf{b}^{(1)} = [0.1, -0.2, 0.0]^T$
- Веса второго слоя: $\mathbf{W}^{(2)} = [0.5, -0.3, 0.7]$
- Смещение второго слоя: $b^{(2)} = 0.1$
- Функция активации: ReLU в скрытом слое, sigmoid в выходном

**Шаг 1:** Вычисляем предактивацию первого скрытого слоя
$$\mathbf{z}^{(1)} = \mathbf{W}^{(1)}\mathbf{x} + \mathbf{b}^{(1)}$$

$$z_1^{(1)} = 0.2 \cdot 0.5 + 0.4 \cdot (-0.3) + 0.1 = 0.1 - 0.12 + 0.1 = 0.08$$
$$z_2^{(1)} = -0.1 \cdot 0.5 + 0.3 \cdot (-0.3) + (-0.2) = -0.05 - 0.09 - 0.2 = -0.34$$
$$z_3^{(1)} = 0.6 \cdot 0.5 + (-0.2) \cdot (-0.3) + 0.0 = 0.3 + 0.06 + 0 = 0.36$$

**Шаг 2:** Применяем ReLU активацию
$$a_1^{(1)} = \text{ReLU}(0.08) = 0.08$$
$$a_2^{(1)} = \text{ReLU}(-0.34) = 0$$
$$a_3^{(1)} = \text{ReLU}(0.36) = 0.36$$

**Шаг 3:** Вычисляем предактивацию выходного слоя
$$z^{(2)} = \mathbf{W}^{(2)}\mathbf{a}^{(1)} + b^{(2)}$$
$$z^{(2)} = 0.5 \cdot 0.08 + (-0.3) \cdot 0 + 0.7 \cdot 0.36 + 0.1 = 0.04 + 0 + 0.252 + 0.1 = 0.392$$

**Шаг 4:** Применяем sigmoid активацию
$$a^{(2)} = \sigma(0.392) = \frac{1}{1 + e^{-0.392}} \approx \frac{1}{1 + 0.676} \approx 0.597$$

**Результат:** Выход сети равен 0.597.

### Влияние архитектуры на обучение

#### Ширина vs Глубина

**Широкие сети (больше нейронов в слое):**
- Больше параметров в каждом слое
- Могут запомнить больше паттернов
- Склонны к переобучению

**Глубокие сети (больше слоев):**
- Иерархическое обучение признаков
- Более эффективное представление
- Проблемы с градиентами

#### Инициализация весов

**Важность правильной инициализации:**
- Слишком большие веса → взрывающиеся градиенты
- Слишком малые веса → исчезающие градиенты
- Все веса одинаковы → симметрия, нейроны не дифференцируются

**Популярные методы:**
- **Xavier/Glorot:** $w \sim \mathcal{N}(0, \frac{2}{n_{in} + n_{out}})$
- **He initialization:** $w \sim \mathcal{N}(0, \frac{2}{n_{in}})$ (для ReLU)
- **LeCun:** $w \sim \mathcal{N}(0, \frac{1}{n_{in}})$

## 7. Forward Pass и Backward Pass: математические основы

### Forward Pass (Прямое распространение)

**Определение:** Forward pass - это процесс вычисления выхода нейронной сети путем последовательного применения матричных операций и функций активации от входного слоя к выходному.

**Математическое описание:**

Для сети с $L$ слоями:

1. **Инициализация:** $\mathbf{a}^{(0)} = \mathbf{x}$ (входные данные)

2. **Для каждого слоя** $l = 1, 2, ..., L$:
   - **Линейное преобразование:** $\mathbf{z}^{(l)} = \mathbf{W}^{(l)}\mathbf{a}^{(l-1)} + \mathbf{b}^{(l)}$
   - **Активация:** $\mathbf{a}^{(l)} = f^{(l)}(\mathbf{z}^{(l)})$

3. **Выход сети:** $\hat{y} = \mathbf{a}^{(L)}$

**Интуиция:** Информация "течет" от входа к выходу, каждый слой преобразует представление данных, делая их более подходящими для решения задачи.

### Backward Pass и Backpropagation

**Определение:** Backward pass (обратное распространение) - это процесс вычисления градиентов функции потерь по всем параметрам сети, используя правило цепочки.

**Цель:** Найти $\frac{\partial L}{\partial \mathbf{W}^{(l)}}$ и $\frac{\partial L}{\partial \mathbf{b}^{(l)}}$ для всех слоев $l$.

### Правило цепочки (Chain Rule)

Для композиции функций $h(x) = f(g(x))$:
$$\frac{dh}{dx} = \frac{df}{dg} \cdot \frac{dg}{dx}$$

В контексте нейронных сетей:
$$\frac{\partial L}{\partial w_{ij}^{(l)}} = \frac{\partial L}{\partial a_i^{(l)}} \cdot \frac{\partial a_i^{(l)}}{\partial z_i^{(l)}} \cdot \frac{\partial z_i^{(l)}}{\partial w_{ij}^{(l)}}$$

### Математический вывод Backpropagation

#### Шаг 1: Градиент по выходному слою

Для функции потерь $L(\hat{y}, y)$:
$$\delta^{(L)} = \frac{\partial L}{\partial \mathbf{z}^{(L)}} = \frac{\partial L}{\partial \mathbf{a}^{(L)}} \odot f'^{(L)}(\mathbf{z}^{(L)})$$

где $\odot$ - поэлементное произведение (произведение Адамара).

#### Шаг 2: Обратное распространение ошибки

Для слоя $l = L-1, L-2, ..., 1$:
$$\delta^{(l)} = (\mathbf{W}^{(l+1)})^T \delta^{(l+1)} \odot f'^{(l)}(\mathbf{z}^{(l)})$$

#### Шаг 3: Вычисление градиентов параметров

**Градиенты весов:**
$$\frac{\partial L}{\partial \mathbf{W}^{(l)}} = \delta^{(l)} (\mathbf{a}^{(l-1)})^T$$

**Градиенты смещений:**
$$\frac{\partial L}{\partial \mathbf{b}^{(l)}} = \delta^{(l)}$$

### Пример ручного расчета Backpropagation

Рассмотрим простую сеть 2-2-1 с квадратичной функцией потерь:

**Дано:**
- Входы: $x_1 = 0.1, x_2 = 0.2$
- Целевое значение: $y = 0.8$
- Веса: $W^{(1)} = \begin{pmatrix} 0.1 & 0.3 \\ 0.2 & 0.4 \end{pmatrix}$, $W^{(2)} = \begin{pmatrix} 0.5 & 0.6 \end{pmatrix}$
- Смещения: $b^{(1)} = \begin{pmatrix} 0.1 \\ 0.2 \end{pmatrix}$, $b^{(2)} = 0.1$
- Функции активации: sigmoid везде
- Функция потерь: $L = \frac{1}{2}(\hat{y} - y)^2$

**Forward Pass:**

*Слой 1:*
$$z_1^{(1)} = 0.1 \cdot 0.1 + 0.3 \cdot 0.2 + 0.1 = 0.01 + 0.06 + 0.1 = 0.17$$
$$z_2^{(1)} = 0.2 \cdot 0.1 + 0.4 \cdot 0.2 + 0.2 = 0.02 + 0.08 + 0.2 = 0.3$$

$$a_1^{(1)} = \sigma(0.17) = \frac{1}{1 + e^{-0.17}} \approx 0.542$$
$$a_2^{(1)} = \sigma(0.3) = \frac{1}{1 + e^{-0.3}} \approx 0.574$$

*Слой 2:*
$$z^{(2)} = 0.5 \cdot 0.542 + 0.6 \cdot 0.574 + 0.1 = 0.271 + 0.344 + 0.1 = 0.715$$
$$\hat{y} = a^{(2)} = \sigma(0.715) \approx 0.672$$

*Функция потерь:*
$$L = \frac{1}{2}(0.672 - 0.8)^2 = \frac{1}{2}(-0.128)^2 \approx 0.0082$$

**Backward Pass:**

*Выходной слой (L=2):*
$$\frac{\partial L}{\partial a^{(2)}} = \hat{y} - y = 0.672 - 0.8 = -0.128$$
$$\frac{\partial a^{(2)}}{\partial z^{(2)}} = \sigma'(0.715) = 0.672(1 - 0.672) = 0.220$$
$$\delta^{(2)} = -0.128 \cdot 0.220 = -0.0282$$

*Скрытый слой (L=1):*
$$\delta_1^{(1)} = W_{11}^{(2)} \cdot \delta^{(2)} \cdot \sigma'(z_1^{(1)}) = 0.5 \cdot (-0.0282) \cdot 0.542(1-0.542) = -0.00341$$
$$\delta_2^{(1)} = W_{12}^{(2)} \cdot \delta^{(2)} \cdot \sigma'(z_2^{(1)}) = 0.6 \cdot (-0.0282) \cdot 0.574(1-0.574) = -0.00413$$

*Градиенты весов:*
$$\frac{\partial L}{\partial W_{11}^{(2)}} = \delta^{(2)} \cdot a_1^{(1)} = -0.0282 \cdot 0.542 = -0.0153$$
$$\frac{\partial L}{\partial W_{12}^{(2)}} = \delta^{(2)} \cdot a_2^{(1)} = -0.0282 \cdot 0.574 = -0.0162$$

$$\frac{\partial L}{\partial W_{11}^{(1)}} = \delta_1^{(1)} \cdot x_1 = -0.00341 \cdot 0.1 = -0.000341$$
$$\frac{\partial L}{\partial W_{12}^{(1)}} = \delta_1^{(1)} \cdot x_2 = -0.00341 \cdot 0.2 = -0.000682$$

### Обновление параметров

С learning rate $\alpha = 0.1$:
$$W_{11}^{(2)} \leftarrow 0.5 - 0.1 \cdot (-0.0153) = 0.5 + 0.00153 = 0.50153$$
$$W_{12}^{(2)} \leftarrow 0.6 - 0.1 \cdot (-0.0162) = 0.6 + 0.00162 = 0.60162$$

### Проблемы и решения

#### 1. Исчезающие градиенты

**Проблема:** Градиенты уменьшаются экспоненциально при распространении назад.

**Причины:**
- Производные sigmoid и tanh < 1
- Произведение многих малых чисел

**Решения:**
- ReLU активации
- Batch Normalization
- Residual connections
- LSTM для RNN

#### 2. Взрывающиеся градиенты

**Проблема:** Градиенты растут экспоненциально.

**Решения:**
- Gradient clipping
- Правильная инициализация весов
- Batch Normalization

#### 3. Седловые точки и локальные минимумы

**Проблема:** Оптимизация может застрять.

**Решения:**
- Momentum
- Adam оптимизатор
- Стохастический градиентный спуск

## 8. PyTorch implementation нейронных сетей

### Введение в PyTorch

PyTorch - это современный фреймворк для глубокого обучения. Он предоставляет:
- **Динамические вычислительные графы** - граф строится во время выполнения
- **Автоматическое дифференцирование** - автоматическое вычисление градиентов
- **GPU ускорение** - эффективные вычисления на GPU
- **Простой и интуитивный API** - похож на NumPy

In [9]:
class SimpleANN(nn.Module):
    """
    Простая полносвязная нейронная сеть для демонстрации основных концепций.
    
    Parameters
    ----------
    input_size : int
        Размер входного слоя
    hidden_size : int
        Размер скрытого слоя
    output_size : int
        Размер выходного слоя
    activation : str, default='relu'
        Функция активации для скрытого слоя
    """
    
    def __init__(self, input_size, hidden_size, output_size, activation='relu'):
        super(SimpleANN, self).__init__()
        
        # Определяем слои
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)
        
        # Выбираем функцию активации
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        else:
            raise ValueError(f"Неподдерживаемая функция активации: {activation}")
    
    def forward(self, x):
        """
        Прямое распространение через сеть.
        
        Parameters
        ----------
        x : torch.Tensor
            Входные данные размера (batch_size, input_size)
            
        Returns
        -------
        torch.Tensor
            Выходные данные размера (batch_size, output_size)
        """
        # Первый слой с активацией
        out = self.linear1(x)
        out = self.activation(out)
        
        # Выходной слой (без активации для регрессии)
        out = self.linear2(out)
        
        return out
    
    def get_parameters_info(self):
        """
        Возвращает информацию о параметрах модели.
        """
        total_params = sum(p.numel() for p in self.parameters())
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        
        print(f"Общее количество параметров: {total_params}")
        print(f"Обучаемых параметров: {trainable_params}")
        
        for name, param in self.named_parameters():
            print(f"{name}: {param.shape}")

# Создание простой модели
model = SimpleANN(input_size=4, hidden_size=8, output_size=2, activation='relu')
print("Архитектура модели:")
print(model)
print("\nИнформация о параметрах:")
model.get_parameters_info()

Архитектура модели:
SimpleANN(
  (linear1): Linear(in_features=4, out_features=8, bias=True)
  (linear2): Linear(in_features=8, out_features=2, bias=True)
  (activation): ReLU()
)

Информация о параметрах:
Общее количество параметров: 58
Обучаемых параметров: 58
linear1.weight: torch.Size([8, 4])
linear1.bias: torch.Size([8])
linear2.weight: torch.Size([2, 8])
linear2.bias: torch.Size([2])


## 9. Обучение на FashionMNIST датасете

FashionMNIST - это датасет изображений одежды размером 28x28 пикселей, содержащий 10 классов. Он является хорошей альтернативой классическому MNIST для демонстрации возможностей нейронных сетей.

In [10]:
# Загрузка и подготовка данных FashionMNIST
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.2860,), (0.3530,))  # Нормализация для FashionMNIST
])

# Загрузка данных
train_dataset = torchvision.datasets.FashionMNIST(
    root='./data', train=True, download=True, transform=transform
)

test_dataset = torchvision.datasets.FashionMNIST(
    root='./data', train=False, download=True, transform=transform
)

# Создание DataLoader'ов
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Названия классов
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print(f"Размер обучающей выборки: {len(train_dataset)}")
print(f"Размер тестовой выборки: {len(test_dataset)}")
print(f"Количество классов: {len(class_names)}")
print(f"Классы: {class_names}")

# Визуализация примеров данных
def visualize_samples(dataset, num_samples=8):
    """
    Визуализация случайных образцов из датасета.
    """
    fig = make_subplots(
        rows=2, cols=4,
        subplot_titles=[f"Класс: {class_names[dataset[i][1]]}" for i in range(num_samples)]
    )
    
    for i in range(num_samples):
        image, label = dataset[i]
        image_np = image.squeeze().numpy()
        
        # Переворачиваем изображение по вертикали для правильного отображения
        image_np = np.flipud(image_np)
        
        row = i // 4 + 1
        col = i % 4 + 1
        
        fig.add_trace(
            go.Heatmap(
                z=image_np,
                colorscale='gray',
                showscale=False
            ),
            row=row, col=col
        )
    
    fig.update_layout(
        title_text="Примеры изображений из FashionMNIST",
        height=400
    )
    
    # Убираем оси для всех подграфиков
    for i in range(1, 3):
        for j in range(1, 5):
            fig.update_xaxes(showticklabels=False, row=i, col=j)
            fig.update_yaxes(showticklabels=False, row=i, col=j)
    
    fig.show()

visualize_samples(train_dataset)

Размер обучающей выборки: 60000
Размер тестовой выборки: 10000
Количество классов: 10
Классы: ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


In [11]:
# Создание модели для классификации FashionMNIST
class FashionMNISTClassifier(nn.Module):
    """
    Нейронная сеть для классификации изображений FashionMNIST.
    
    Parameters
    ----------
    hidden_sizes : list
        Список размеров скрытых слоев
    num_classes : int, default=10
        Количество классов для классификации
    dropout_rate : float, default=0.2
        Вероятность dropout для регуляризации
    """
    
    def __init__(self, hidden_sizes=[512, 256], num_classes=10, dropout_rate=0.2):
        super(FashionMNISTClassifier, self).__init__()
        
        # Размер входного изображения: 28x28 = 784 пикселя
        self.input_size = 28 * 28
        
        # Создаем список слоев
        layers = []
        input_size = self.input_size
        
        # Добавляем скрытые слои
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(input_size, hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            input_size = hidden_size
        
        # Выходной слой (без активации - используем CrossEntropyLoss)
        layers.append(nn.Linear(input_size, num_classes))
        
        # Создаем последовательную модель
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        """
        Прямое распространение через сеть.
        
        Parameters
        ----------
        x : torch.Tensor
            Входные изображения размера (batch_size, 1, 28, 28)
            
        Returns
        -------
        torch.Tensor
            Логиты для каждого класса размера (batch_size, num_classes)
        """
        # Преобразуем изображения в плоский вектор
        x = x.view(x.size(0), -1)  # (batch_size, 784)
        
        # Пропускаем через сеть
        return self.network(x)
    
    def get_num_parameters(self):
        """Возвращает общее количество параметров модели."""
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

# Создание модели
model = FashionMNISTClassifier(
    hidden_sizes=[512, 256, 128], 
    num_classes=10, 
    dropout_rate=0.3
).to(device)

print("Архитектура модели:")
print(model)
print(f"\nОбщее количество параметров: {model.get_num_parameters():,}")
print(f"Модель размещена на: {device}")

Архитектура модели:
FashionMNISTClassifier(
  (network): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=256, out_features=128, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=128, out_features=10, bias=True)
  )
)

Общее количество параметров: 567,434
Модель размещена на: cpu


In [12]:
# Настройка обучения
def train_model(model, train_loader, test_loader, num_epochs=5, learning_rate=0.001):
    """
    Обучение модели на данных FashionMNIST.
    
    Parameters
    ----------
    model : nn.Module
        Модель для обучения
    train_loader : DataLoader
        Загрузчик обучающих данных
    test_loader : DataLoader
        Загрузчик тестовых данных
    num_epochs : int
        Количество эпох обучения
    learning_rate : float
        Скорость обучения
        
    Returns
    -------
    dict
        Словарь с историей обучения (loss и accuracy)
    """
    # Определяем функцию потерь и оптимизатор
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Для сохранения истории обучения
    history = {
        'train_loss': [],
        'train_acc': [],
        'test_loss': [],
        'test_acc': []
    }
    
    print("Начало обучения...")
    print("=" * 50)
    
    for epoch in range(num_epochs):
        # === ОБУЧЕНИЕ ===
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            
            # Обнуляем градиенты
            optimizer.zero_grad()
            
            # Прямой проход
            output = model(data)
            loss = criterion(output, target)
            
            # Обратный проход
            loss.backward()
            optimizer.step()
            
            # Статистика
            train_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            train_total += target.size(0)
            train_correct += (predicted == target).sum().item()
            
            # Прогресс каждые 100 батчей
            if batch_idx % 100 == 0:
                print(f'Эпоха {epoch+1}/{num_epochs}, Батч {batch_idx}/{len(train_loader)}, '
                      f'Loss: {loss.item():.4f}')
        
        # === ВАЛИДАЦИЯ ===
        model.eval()
        test_loss = 0.0
        test_correct = 0
        test_total = 0
        
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += criterion(output, target).item()
                
                _, predicted = torch.max(output.data, 1)
                test_total += target.size(0)
                test_correct += (predicted == target).sum().item()
        
        # Вычисляем средние значения
        avg_train_loss = train_loss / len(train_loader)
        avg_test_loss = test_loss / len(test_loader)
        train_acc = 100. * train_correct / train_total
        test_acc = 100. * test_correct / test_total
        
        # Сохраняем историю
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_acc)
        history['test_loss'].append(avg_test_loss)
        history['test_acc'].append(test_acc)
        
        # Выводим результаты эпохи
        print(f'Эпоха {epoch+1}/{num_epochs}:')
        print(f'  Обучение  - Loss: {avg_train_loss:.4f}, Accuracy: {train_acc:.2f}%')
        print(f'  Валидация - Loss: {avg_test_loss:.4f}, Accuracy: {test_acc:.2f}%')
        print("-" * 50)
    
    print("Обучение завершено!")
    return history

# Запуск обучения
history = train_model(
    model=model,
    train_loader=train_loader,
    test_loader=test_loader,
    num_epochs=5,
    learning_rate=0.001
)

Начало обучения...
Эпоха 1/5, Батч 0/469, Loss: 2.2968
Эпоха 1/5, Батч 100/469, Loss: 0.5367
Эпоха 1/5, Батч 200/469, Loss: 0.4050
Эпоха 1/5, Батч 300/469, Loss: 0.3923
Эпоха 1/5, Батч 400/469, Loss: 0.4318
Эпоха 1/5:
  Обучение  - Loss: 0.5644, Accuracy: 79.64%
  Валидация - Loss: 0.4574, Accuracy: 83.03%
--------------------------------------------------
Эпоха 2/5, Батч 0/469, Loss: 0.4579
Эпоха 2/5, Батч 100/469, Loss: 0.4937
Эпоха 2/5, Батч 200/469, Loss: 0.3053
Эпоха 2/5, Батч 300/469, Loss: 0.3119
Эпоха 2/5, Батч 400/469, Loss: 0.5353
Эпоха 2/5:
  Обучение  - Loss: 0.4086, Accuracy: 85.34%
  Валидация - Loss: 0.3771, Accuracy: 86.12%
--------------------------------------------------
Эпоха 3/5, Батч 0/469, Loss: 0.3184
Эпоха 3/5, Батч 100/469, Loss: 0.3194
Эпоха 3/5, Батч 200/469, Loss: 0.3334
Эпоха 3/5, Батч 300/469, Loss: 0.3145
Эпоха 3/5, Батч 400/469, Loss: 0.4132
Эпоха 3/5:
  Обучение  - Loss: 0.3735, Accuracy: 86.53%
  Валидация - Loss: 0.3769, Accuracy: 86.01%
------------

In [13]:
# Визуализация процесса обучения
def plot_training_history(history):
    """
    Визуализация истории обучения модели.
    
    Parameters
    ----------
    history : dict
        Словарь с историей обучения
    """
    epochs = list(range(1, len(history['train_loss']) + 1))
    
    # Создание подграфиков
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=['Функция потерь (Loss)', 'Точность (Accuracy)'],
        horizontal_spacing=0.15
    )
    
    # График функции потерь
    fig.add_trace(
        go.Scatter(
            x=epochs, 
            y=history['train_loss'],
            mode='lines+markers',
            name='Обучение',
            line=dict(color='blue', width=2),
            marker=dict(size=6)
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=epochs, 
            y=history['test_loss'],
            mode='lines+markers',
            name='Валидация',
            line=dict(color='red', width=2),
            marker=dict(size=6)
        ),
        row=1, col=1
    )
    
    # График точности
    fig.add_trace(
        go.Scatter(
            x=epochs, 
            y=history['train_acc'],
            mode='lines+markers',
            name='Обучение',
            line=dict(color='blue', width=2),
            marker=dict(size=6),
            showlegend=False
        ),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Scatter(
            x=epochs, 
            y=history['test_acc'],
            mode='lines+markers',
            name='Валидация',
            line=dict(color='red', width=2),
            marker=dict(size=6),
            showlegend=False
        ),
        row=1, col=2
    )
    
    # Настройка осей
    fig.update_xaxes(title_text="Эпоха", row=1, col=1)
    fig.update_yaxes(title_text="Loss", row=1, col=1)
    fig.update_xaxes(title_text="Эпоха", row=1, col=2)
    fig.update_yaxes(title_text="Accuracy (%)", row=1, col=2)
    
    fig.update_layout(
        title_text="История обучения нейронной сети на FashionMNIST",
        title_x=0.5,
        height=400,
        legend=dict(x=0.02, y=0.98)
    )
    
    fig.show()

# Построение графиков
plot_training_history(history)

# Выводим финальные результаты
final_train_acc = history['train_acc'][-1]
final_test_acc = history['test_acc'][-1]
final_train_loss = history['train_loss'][-1]
final_test_loss = history['test_loss'][-1]

print("ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ:")
print("=" * 40)
print(f"Точность на обучающей выборке: {final_train_acc:.2f}%")
print(f"Точность на тестовой выборке:  {final_test_acc:.2f}%")
print(f"Потери на обучающей выборке:  {final_train_loss:.4f}")
print(f"Потери на тестовой выборке:   {final_test_loss:.4f}")
print("=" * 40)

ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ:
Точность на обучающей выборке: 87.84%
Точность на тестовой выборке:  87.48%
Потери на обучающей выборке:  0.3346
Потери на тестовой выборке:   0.3446


In [14]:
# Анализ предсказаний модели
def analyze_predictions(model, test_loader, class_names, num_samples=16):
    """
    Анализ предсказаний модели на тестовых данных.
    
    Parameters
    ----------
    model : nn.Module
        Обученная модель
    test_loader : DataLoader
        Загрузчик тестовых данных
    class_names : list
        Названия классов
    num_samples : int
        Количество примеров для отображения
    """
    model.eval()
    
    # Получаем батч тестовых данных
    data_iter = iter(test_loader)
    images, labels = next(data_iter)
    images, labels = images.to(device), labels.to(device)
    
    # Делаем предсказания
    with torch.no_grad():
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        # Вычисляем вероятности с помощью softmax
        probabilities = F.softmax(outputs, dim=1)
    
    # Визуализация результатов
    fig = make_subplots(
        rows=4, cols=4,
        subplot_titles=[
            f"Истина: {class_names[labels[i]][:8]}<br>"  # Сокращаем длинные названия
            f"Предск.: {class_names[predicted[i]][:8]}<br>"
            f"Ув.: {probabilities[i][predicted[i]]:.2f}"
            for i in range(num_samples)
        ],
        vertical_spacing=0.12,  # Увеличиваем вертикальный отступ
        horizontal_spacing=0.08  # Увеличиваем горизонтальный отступ
    )
    
    for i in range(num_samples):
        image = images[i].cpu().squeeze().numpy()
        
        # Переворачиваем изображение по вертикали для правильного отображения
        image = np.flipud(image)
        
        row = i // 4 + 1
        col = i % 4 + 1
        
        fig.add_trace(
            go.Heatmap(
                z=image,
                colorscale='gray',
                showscale=False,
                hovertemplate=f"Истинный класс: {class_names[labels[i]]}<br>" +
                             f"Предсказанный класс: {class_names[predicted[i]]}<br>" +
                             f"Уверенность: {probabilities[i][predicted[i]]:.3f}<extra></extra>"
            ),
            row=row, col=col
        )
    
    # Настройка осей (убираем подписи)
    for i in range(1, 5):
        for j in range(1, 5):
            fig.update_xaxes(showticklabels=False, showgrid=False, row=i, col=j)
            fig.update_yaxes(showticklabels=False, showgrid=False, row=i, col=j)
    
    fig.update_layout(
        title_text="Примеры предсказаний модели на тестовых данных",
        title_x=0.5,
        title_font=dict(size=16),  # Размер заголовка
        height=700,  # Увеличиваем высоту
        showlegend=False,
        font=dict(size=9),  # Уменьшаем размер шрифта
        margin=dict(t=100, b=50, l=50, r=50)  # Добавляем отступы
    )
    
    # Обновляем подзаголовки
    fig.update_annotations(font_size=9)  # Уменьшаем размер подзаголовков
    
    fig.show()
    
    # Статистика по классам
    correct_by_class = torch.zeros(len(class_names))
    total_by_class = torch.zeros(len(class_names))
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, pred = torch.max(output, 1)
            
            for i in range(len(class_names)):
                mask = (target == i)
                total_by_class[i] += mask.sum().item()
                correct_by_class[i] += (pred[mask] == target[mask]).sum().item()
    
    # Вычисляем точность по классам
    class_accuracy = correct_by_class / total_by_class * 100
    
    print("\nТОЧНОСТЬ ПО КЛАССАМ:")
    print("=" * 50)
    for i, class_name in enumerate(class_names):
        print(f"{class_name:15}: {class_accuracy[i]:.2f}% ({int(correct_by_class[i])}/{int(total_by_class[i])})")
    print("=" * 50)
    print(f"{'Средняя точность':15}: {class_accuracy.mean():.2f}%")

# Анализ предсказаний
analyze_predictions(model, test_loader, class_names, num_samples=16)


ТОЧНОСТЬ ПО КЛАССАМ:
T-shirt/top    : 81.60% (816/1000)
Trouser        : 97.00% (970/1000)
Pullover       : 82.30% (823/1000)
Dress          : 88.10% (881/1000)
Coat           : 73.20% (732/1000)
Sandal         : 95.50% (955/1000)
Shirt          : 69.80% (698/1000)
Sneaker        : 96.40% (964/1000)
Bag            : 97.10% (971/1000)
Ankle boot     : 93.80% (938/1000)
Средняя точность: 87.48%


In [15]:
# Интерактивные эксперименты с архитектурой
def interactive_architecture_experiment(hidden_layers='[512, 256]', dropout_rate=0.3, 
                                       learning_rate=0.001, num_epochs=3):
    """
    Интерактивный эксперимент с различными архитектурами сети.
    
    Parameters
    ----------
    hidden_layers : str
        Строковое представление списка размеров скрытых слоев
    dropout_rate : float
        Вероятность dropout
    learning_rate : float
        Скорость обучения
    num_epochs : int
        Количество эпох для обучения
    """
    try:
        # Парсим строку со скрытыми слоями
        hidden_sizes = eval(hidden_layers)
        
        print(f"ЭКСПЕРИМЕНТ С АРХИТЕКТУРОЙ:")
        print(f"Скрытые слои: {hidden_sizes}")
        print(f"Dropout: {dropout_rate}")
        print(f"Learning rate: {learning_rate}")
        print(f"Эпохи: {num_epochs}")
        print("=" * 50)
        
        # Создаем новую модель
        experiment_model = FashionMNISTClassifier(
            hidden_sizes=hidden_sizes,
            num_classes=10,
            dropout_rate=dropout_rate
        ).to(device)
        
        print(f"Параметров в модели: {experiment_model.get_num_parameters():,}")
        
        # Обучаем модель
        exp_history = train_model(
            model=experiment_model,
            train_loader=train_loader,
            test_loader=test_loader,
            num_epochs=num_epochs,
            learning_rate=learning_rate
        )
        
        # Визуализируем результаты
        plot_training_history(exp_history)
        
        # Финальная точность
        final_acc = exp_history['test_acc'][-1]
        print(f"\nФинальная точность: {final_acc:.2f}%")
        
        return experiment_model, exp_history
        
    except Exception as e:
        print(f"Ошибка в эксперименте: {e}")
        print("Пример правильного формата: [512, 256, 128]")

# Создание интерактивного виджета
print("ИНТЕРАКТИВНЫЕ ЭКСПЕРИМЕНТЫ С АРХИТЕКТУРОЙ")
print("Настройте параметры и нажмите кнопку 'Run Interact' для запуска эксперимента:")
print()

interact_manual(interactive_architecture_experiment,
                hidden_layers=widgets.Text(
                    value='[512, 256]',
                    description='Скрытые слои:',
                    style={'description_width': 'initial'}
                ),
                dropout_rate=widgets.FloatSlider(
                    value=0.3,
                    min=0.0,
                    max=0.8,
                    step=0.1,
                    description='Dropout:',
                    style={'description_width': 'initial'}
                ),
                learning_rate=widgets.FloatLogSlider(
                    value=0.001,
                    base=10,
                    min=-4,
                    max=-1,
                    step=0.1,
                    description='Learning rate:',
                    style={'description_width': 'initial'}
                ),
                num_epochs=widgets.IntSlider(
                    value=3,
                    min=1,
                    max=10,
                    step=1,
                    description='Эпохи:',
                    style={'description_width': 'initial'}
                ));

ИНТЕРАКТИВНЫЕ ЭКСПЕРИМЕНТЫ С АРХИТЕКТУРОЙ
Настройте параметры и нажмите кнопку 'Run Interact' для запуска эксперимента:



interactive(children=(Text(value='[512, 256]', description='Скрытые слои:', style=DescriptionStyle(description…

## 10. TO-DO

### Общие требования
- Все решения должны быть подробно прокомментированы
- Используйте NumPy docstring формат для функций
- Код должен быть разбит на логические блоки

### Задание 1: Реализация функций активации (20%)

**Подзадание 1.1 (10%):** Реализуйте функцию активации ELU (Exponential Linear Unit):

$$\text{ELU}(x) = \begin{cases} 
x & \text{если } x > 0 \\
\alpha(e^x - 1) & \text{если } x \leq 0
\end{cases}$$

где $\alpha = 1.0$ по умолчанию.

**Требования:**
- Реализуйте функцию `elu(z, alpha=1.0)` и её производную `elu_derivative(z, alpha=1.0)`
- Добавьте docstring с описанием параметров и возвращаемых значений
- Протестируйте на массиве `[-2, -1, 0, 1, 2]`

**Подзадание 1.2 (10%):** Создайте интерактивную визуализацию ELU функции:
- Сравните ELU с ReLU и Leaky ReLU
- Добавьте слайдер для изменения параметра alpha (от 0.1 до 2.0)
- Покажите как alpha влияет на форму функции для отрицательных значений

### Задание 2: Ручные вычисления (25%)

**Подзадание 2.1 (15%):** Выполните полный forward pass для сети 3-2-1:

**Дано:**
- Входы: $x_1 = 0.2, x_2 = -0.1, x_3 = 0.5$
- Веса первого слоя: $W^{(1)} = \begin{pmatrix} 0.1 & 0.2 & -0.1 \\ 0.3 & -0.2 & 0.4 \end{pmatrix}$
- Смещения первого слоя: $b^{(1)} = [0.1, -0.1]^T$
- Веса второго слоя: $W^{(2)} = [0.5, -0.3]$
- Смещение второго слоя: $b^{(2)} = 0.2$
- Функции активации: ReLU в скрытом слое, sigmoid в выходном

**Требования:**
- Покажите все промежуточные вычисления
- Округляйте до 4 знаков после запятой
- Оформите решение в markdown ячейке с использованием LaTeX

**Подзадание 2.2 (10%):** Для той же сети выполните один шаг backward pass:
- Целевое значение: $y = 0.8$
- Функция потерь: $L = \frac{1}{2}(\hat{y} - y)^2$
- Вычислите $\frac{\partial L}{\partial W^{(2)}}$ и $\frac{\partial L}{\partial b^{(2)}}$

### Задание 3: Программная реализация (30%)

**Подзадание 3.1 (20%):** Создайте класс `CustomANN` с следующими возможностями:

```python
class CustomANN(nn.Module):
    def __init__(self, layer_sizes, activation='relu', output_activation=None):
        # Ваша реализация
        pass
    
    def forward(self, x):
        # Ваша реализация
        pass
    
    def count_parameters(self):
        # Подсчет общего количества параметров
        pass
    
    def get_layer_outputs(self, x):
        # Возвращает выходы всех слоев (для анализа)
        pass
```

**Требования:**
- `layer_sizes` - список размеров слоев, например [784, 128, 64, 10]
- Поддержка различных функций активации
- Метод для подсчета параметров
- Метод для получения выходов промежуточных слоев

**Подзадание 3.2 (10%):** Протестируйте модель на синтетических данных:
- Создайте датасет для задачи классификации (4 класса)
- Обучите модель архитектуры 2-4-1 в течение 100 эпох
- Постройте график изменения loss во время обучения
- Покажите финальные предсказания модели

### Задание 4: Эксперименты с архитектурой (25%)

**Подзадание 4.1 (15%):** Исследование влияния глубины сети:

Сравните производительность следующих архитектур на FashionMNIST:
1. Неглубокая: 784-256-10
2. Средняя: 784-128-64-10  
3. Глубокая: 784-128-64-32-16-10

**Требования:**
- Обучите каждую модель на 5 эпохах
- Используйте одинаковые гиперпараметры (learning rate, batch size)
- Сравните: время обучения, точность на валидации, количество параметров
- Создайте таблицу результатов и график сравнения точности

**Подзадание 4.2 (10%):** Влияние функций активации:

Сравните ReLU, Sigmoid и Tanh на архитектуре 784-128-64-10:
- Обучите три модели с разными функциями активации
- Постройте графики обучения (loss и accuracy)
- Проанализируйте различия в скорости сходимости
- Объясните результаты с теоретической точки зрения